In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

import matplotlib.patches as patches
import matplotlib

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])    

In [4]:
#result_all[result_all['step']!=1]

In [5]:
r2_df=pd.read_csv('data/r2.tsv',sep='\t',index_col=0)
#r2_df=pd.DataFrame(index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)

In [9]:
#result_all

In [10]:
result_all=pd.read_csv('data/result_concat_1_top_all.tsv',sep='\t')
len(result_all['marker_name']),len(result_all[result_all['P']<5e-8]['marker_name'].unique())

(168, 50)

In [7]:
len(result_all[result_all['P']<5e-8]['marker_name'].unique())

50

In [11]:
top_markers=pd.Series(result_all[result_all['P']<5e-8]['marker_name'].unique())
len(top_markers)

50

In [12]:
top_markers_SNP=top_markers[~((top_markers.str.slice(stop=3)=='HLA')|(top_markers.str.slice(stop=3)=='AA_')|(top_markers.str.slice(stop=3)=='SNP'))]
len(top_markers_SNP)

38

In [13]:
print(1)

1


In [14]:
for marker_name in top_markers_SNP.values:
    if marker_name in r2_df.columns:
        print(marker_name,'continue')
        continue
    else:
        print(marker_name,'test')
    marker_dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(marker_name)
    print(marker_name)
    r2_list=[]
    for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim.iterrows()):
        r2=pearsonr(marker_dosage,plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP))[0]**2
        r2_list.append(r2)
    
    r2_df[marker_name]=r2_list
    
    #r2_df_temp=pd.DataFrame(r2_list,index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)
    #r2_df['here']=marker_name
    #r2_df=r2_df.sort_values(0,ascending=False)
    
    #r2_df_dict[marker_name]=r2_df

6:34172055_C/A continue
6:34205465_G/GGAGCCC continue
6:34215221_G/A continue
AX-15354567 continue
6:32632237_C/T continue
6:31351530_T/C continue
6:34178764_A/C continue
6:33076268_T/C continue
6:29896680_A/G continue
6:29190624_G/A continue
6:30737591_G/C continue
6:31237858_T/C continue
AX-11365330 continue
6:31371397_C/T continue
6:31358559_C/T continue
6:32377408_T/C continue
6:34810172_T/C continue
6:34207474_T/TAA continue
6:34967349_G/A continue
AX-11435484 continue
6:32149801_C/T continue
6:29698050_A/G continue
6:34231005_T/C continue
6:28749731_TA/T continue
6:31538497_T/A continue
6:32638199_C/G continue
AX-41961031 continue
6:32324081_A/AT continue
AX-11397307 continue
6:34205822_T/C continue
6:32109294_A/G continue
6:29938478_A/G continue
6:34185749_C/T continue
6:31428911_A/G continue
6:34234974_A/G continue
6:32569297_A/G continue
6:28861320_A/G continue
6:30990019_A/G continue


snp
6:28000361_T/C               0.003062
6:28000610_C/A               0.003059
6:28001003_A/C               0.002887
6:28001033_T/C               0.014531
6:28001610_T/C               0.014504
6:28002045_G/A               0.003117
6:28002139_G/A               0.004478
6:28002388_G/A               0.005030
6:28002818_C/T               0.014531
6:28002853_T/C               0.014531
6:28002963_A/G               0.003059
6:28003195_CTG/C             0.037027
6:28003205_G/A               0.005026
6:28003222_C/T               0.003765
6:28003275_T/G               0.003062
6:28004059_T/C               0.009317
6:28004078_A/G               0.012635
6:28004133_G/T               0.000319
6:28004134_G/A               0.003062
6:28004381_A/G               0.000489
6:28004835_A/G               0.000422
6:28005112_A/G               0.003061
6:28006042_A/G               0.003075
6:28006650_A/G               0.014641
6:28007357_A/C               0.014640
6:28007406_AG/A              0.009334
6:280074

In [26]:
r2_df.to_csv('data/r2.tsv',sep='\t')

In [27]:
#pd.read_csv('data/result_concat_2_top_r2.tsv',sep='\t',index_col=0)

In [28]:
pd.read_csv('data/r2.tsv',sep='\t')

snp  6:31018788_T/G   AX-11396301  \
0                 6:28000361_T/C    3.666156e-03  4.260964e-03   
1                 6:28000610_C/A    3.675666e-03  4.272276e-03   
2                 6:28001003_A/C    3.605884e-03  4.311554e-03   
3                 6:28001033_T/C    1.225358e-02  1.596598e-02   
4                 6:28001610_T/C    1.221706e-02  1.593404e-02   
5                 6:28002045_G/A    3.709945e-03  4.308135e-03   
6                 6:28002139_G/A    2.770758e-03  6.110681e-03   
7                 6:28002388_G/A    6.917218e-03  1.116468e-02   
8                 6:28002818_C/T    1.225358e-02  1.596598e-02   
9                 6:28002853_T/C    1.225358e-02  1.596598e-02   
10                6:28002963_A/G    3.675666e-03  4.272276e-03   
11              6:28003195_CTG/C    3.952001e-03  6.771249e-03   
12                6:28003205_G/A    7.249565e-03  1.146207e-02   
13                6:28003222_C/T    5.332718e-03  6.231887e-03   
14                6:28003275_T/G    3.666156e-03  4.260964e-03   
15                6:28004059_T/C    6.656534e-03  9.246612e-03   
16                6:28004078_A/G    8.243434e-05  1.030088e-04   
17                6:28004133_G/T    8.288888e-03  7.477143e-03   
18                6:28004134_G/A    3.666156e-03  4.260964e-03   
19                6:28004381_A/G    4.083467e-03  4.650468e-03   
20                6:28004835_A/G    1.054675e-02  9.267093e-03   
21                6:28005112_A/G    3.662767e-03  4.257546e-03   
22                6:28006042_A/G    3.166726e-03  3.621915e-03   
23                6:28006650_A/G    1.254291e-02  1.635017e-02   
24                6:28007357_A/C    1.251929e-02  1.632969e-02   
25               6:28007406_AG/A    6.833025e-03  9.454392e-03   
26                6:28007419_C/T    3.665823e-03  4.247301e-03   
27              6:28007595_CTT/C    2.070761e-03  2.824015e-03   
28                6:28007911_A/C    8.254178e-04  9.737300e-04   
29                6:28007930_C/A    2.461366e-04  8.398042e-05   
...                          ...             ...           ...   
70957            6:34982237_TC/T    1.961764e-03  1.991792e-03   
70958             6:34982599_C/T    6.680373e-04  9.155512e-04   
70959            6:34983275_C/CA    2.395850e-04  1.591269e-04   
70960             6:34984037_A/C    6.715169e-04  9.194409e-04   
70961            6:34984556_A/AG    1.500659e-04  9.213282e-05   
70962             6:34984558_A/G    7.023714e-07  4.099215e-06   
70963           6:34984712_CTT/C    7.329160e-04  7.536175e-04   
70964         6:34985133_TTCTC/T    3.785013e-05  4.594090e-05   
70965             6:34985563_G/A    1.835402e-05  8.903616e-06   
70966             6:34985985_G/A    1.270089e-03  1.532635e-03   
70967            6:34986340_CA/C    6.646096e-06  8.648749e-07   
70968             6:34986591_G/C    1.142641e-05  2.673394e-06   
70969             6:34986717_A/G    4.508901e-05  6.150380e-05   
70970             6:34988391_C/A    1.223865e-03  1.191862e-03   
70971             6:34988951_A/C    7.517865e-04  9.948555e-04   
70972             6:34989278_C/T    6.322573e-05  5.817801e-05   
70973             6:34989598_A/G    6.322573e-05  5.817801e-05   
70974            6:34989761_T/TA    3.903436e-04  2.634513e-04   
70975             6:34990203_G/C    9.724513e-05  8.865797e-05   
70976             6:34990968_C/T    1.744302e-04  1.036176e-04   
70977  6:34993233_T/TCCCTTGCTGCC    9.374983e-05  1.213411e-04   
70978          6:34994293_TTGA/T    1.891953e-04  1.170467e-04   
70979             6:34994686_A/G    1.092479e-05  2.146473e-06   
70980             6:34995973_T/C    4.544221e-05  3.899058e-05   
70981             6:34996132_A/T    1.203886e-04  1.630915e-04   
70982             6:34997188_C/T    1.642962e-04  9.499229e-05   
70983             6:34997259_G/A    1.640890e-04  2.135870e-04   
70984             6:34997606_T/A    1.149787e-03  1.356930e-03   
70985             6:34998418_G/A    2.075309e-04  2.554163e-04   
709